In [500]:
import requests
from pprint import pprint
from itertools import izip
from pandas import DataFrame
from collections import defaultdict

In [501]:
url = 'http://stats.nba.com/stats/playerdashptshotlog?'+ \
    'DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&' + \
    'Location=&Month=0&OpponentTeamID=0&Outcome=&Period=0&' + \
    'PlayerID={player_id}&Season=2014-15&SeasonSegment=&' + \
    'SeasonType=Regular+Season&TeamID=0&VsConference=&VsDivision='

# Stephen Curry
PLAYER_ID = 201939

# James Harden
PLAYER_ID = 201935

resp = requests.get(url.format(player_id=PLAYER_ID))
data = resp.json()

In [502]:
shot_data = data['resultSets'][0]['rowSet']
headers = [h.lower() for h in data['resultSets'][0]['headers']]
shots = [dict(izip(headers, s)) for s in shot_data]
print "Total FGA: ", len(shots)
makes_misses = defaultdict(lambda: dict(made=0, missed=0, total=0, close_def_dist=0.0))

# fill out dictionary
for shot in shots:
    makes_misses[shot['closest_defender']][shot['shot_result']] += 1
    makes_misses[shot['closest_defender']]['close_def_dist'] += shot['close_def_dist']
    
# reduce to standard dictionary type and compute totals
makes_misses = dict(izip(makes_misses.keys(), map(dict, makes_misses.values())))
for s in makes_misses.itervalues():
    s['total'] = s['made'] + s['missed']
    s['close_def_dist_avg'] = s['close_def_dist']/s['total']

Total FGA:  1443


In [503]:
print headers
df = DataFrame(shots, columns=headers)
print df.head(3)
shot_df = DataFrame(makes_misses)
#print shot_df.head()

[u'game_id', u'matchup', u'location', u'w', u'final_margin', u'shot_number', u'period', u'game_clock', u'shot_clock', u'dribbles', u'touch_time', u'shot_dist', u'pts_type', u'shot_result', u'closest_defender', u'closest_defender_player_id', u'close_def_dist', u'fgm', u'pts']
      game_id                     matchup location  w  final_margin  \
0  0021401219  APR 15, 2015 - HOU vs. UTA        H  W            26   
1  0021401219  APR 15, 2015 - HOU vs. UTA        H  W            26   
2  0021401219  APR 15, 2015 - HOU vs. UTA        H  W            26   

   shot_number  period game_clock  shot_clock  dribbles  touch_time  \
0            1       1      11:00        19.5         4         3.8   
1            2       1       4:41        15.0         6         8.0   
2            3       1       1:41        17.0         1         1.5   

   shot_dist  pts_type shot_result closest_defender  \
0        8.2         2      missed      Exum, Dante   
1        3.2         2        made    Evans,

In [504]:
shot_df_t = shot_df.T
print shot_df_t.head()
shot_df_t['fg_pct'] = shot_df_t['made'].astype('float') / shot_df_t['total'].astype('float')
shot_df_t_sorted = shot_df_t.sort('fg_pct', ascending=False)
shot_df_t_sorted[shot_df_t_sorted['total'] > 10].sort('total', ascending=False).head(10)

                    close_def_dist  close_def_dist_avg  made  missed  total
Acy, Quincy                    2.6            2.600000     0       1      1
Afflalo, Arron                47.3            2.956250     8       8     16
Aldrich, Cole                  4.4            4.400000     1       0      1
Aldridge, LaMarcus            15.5            5.166667     1       2      3
Allen, Tony                   51.1            4.645455     4       7     11


,close_def_dist,close_def_dist_avg,made,missed,total,fg_pct
"McLemore, Ben",145.2,3.630000,15,25,40,0.375000
"Tucker, PJ",125.5,3.691176,12,22,34,0.352941
"Thompson, Klay",108.9,3.630000,11,19,30,0.366667
"Redick, JJ",113.7,3.920690,9,20,29,0.310345
"Green, Danny",73.2,3.182609,9,14,23,0.391304
"Wiggins, Andrew",88.6,4.027273,10,12,22,0.454545
"Pondexter, Quincy",75.1,4.172222,6,12,18,0.333333
"Butler, Jimmy",63.1,3.505556,8,10,18,0.444444
"Matthews, Wesley",62.4,3.466667,12,6,18,0.666667
"Johnson, Wesley",56.9,3.556250,9,7,16,0.562500


In [505]:
shot_defender = df[['closest_defender', 'shot_result']]
sd = shot_defender.groupby(['shot_result', 'closest_defender'])
#for k, v in sd:
#    print k, v